In [1]:
import sys
sys.path.append('/home/nicholasjesperson/Documents/School/Comp4780/Data_Augmentation/Data Augmentation/paper/AugmenTRAJ')

In [2]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.core.Datasets import Datasets
from ptrail.preprocessing.statistics import Statistics

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


# Load the Starkey dataset.
starkey = Datasets.load_starkey()
starkey.head()

------------------------ Dataset Facts ------------------------------

Number of unique Trajectories in the data: 253
Number of points in the data: 287136
Dataset time range: 1196 days 22:51:45
Datatype of the DataFrame: <class 'ptrail.core.TrajectoryDF.PTRAILDataFrame'>
Dataset Bounding Box: (45.18896978643169, -118.61020848239596, 45.314545642992, -118.50455596234036)

---------------------------------------------------------------------


lat         lon  StarkeyTime    GMDate  \
traj_id   DateTime                                                            
880109D01 1995-04-13 13:40:06  45.239682 -118.533204    229902006  21:40:06   
          1995-04-15 12:16:15  45.250521 -118.530438    230069775  20:16:15   
          1995-04-15 21:39:38  45.247943 -118.541455    230103578  05:39:38   
          1995-04-16 03:32:14  45.247429 -118.539530    230124734  11:32:14   
          1995-04-16 04:08:28  45.247117 -118.542579    230126908  12:08:28   

                                 GMTime   LocDate   LocTime  RadNum  Species  \
traj_id   DateTime                                                             
880109D01 1995-04-13 13:40:06  19950413  19950413  13:40:06     409        0   
          1995-04-15 12:16:15  19950415  19950415  12:16:15     409        0   
          1995-04-15 21:39:38  19950416  19950415  21:39:38     409        0   
          1995-04-16 03:32:14  19950416  19950416  03:32:14     409        0   
          1995-04-16 04:08:28  19950416  19950416  04:08:28     409        0   

                                 UTME     UTMN  Year  Grensunr  Grensuns  \
traj_id   DateTime                                                         
880109D01 1995-04-13 13:40:06  379662  5010734    95  13:13:00  02:39:00   
          1995-04-15 12:16:15  379895  5011927    95  13:09:00  02:41:00   
          1995-04-15 21:39:38  379039  5011656    95  13:07:00  02:43:00   
          1995-04-16 03:32:14  379188  5011581    95  13:07:00  02:43:00   
          1995-04-16 04:08:28  378938  5011567    95  13:07:00  02:43:00   

                               Obswt  
traj_id   DateTime                    
880109D01 1995-04-13 13:40:06   1.47  
          1995-04-15 12:16:15   1.59  
          1995-04-15 21:39:38   1.34  
          1995-04-16 03:32:14   1.50  
          1995-04-16 04:08:28   1.34

In [3]:
stats_df = Statistics.generate_kinematic_stats(dataframe=starkey,
                                               target_col_name='Species')
pivoted = Statistics.pivot_stats_df(dataframe=stats_df,
                                    target_col_name='Species')

In [4]:
seeds = [14159,26535,89793] #Model states
split_states = [526, 14855, 62586, 324578, 74125, 32155, 7, 4852, 94652, 33333]
text_file = open("baselineStarkey.txt", "w")
for s in seeds:
    model = RandomForestClassifier(random_state=s)
    for state in split_states:
        X_train, X_test, y_train, y_test = train_test_split(
        pivoted.drop(columns=['Species']), pivoted['Species'], test_size=0.2, random_state=state)

        model.fit(X_train, y_train)
        test_predict = model.predict(X_test)
        performance_val = f1_score(y_test, test_predict, average='weighted')
        print(performance_val)

        text_file.writelines(str(performance_val) + '\n')
    
text_file.close()

0.8534444006858171
0.9451267686561803
0.8363261093911248
0.9185635940199833
0.960656468004961
0.9622782446311858
0.9175282793594526
0.9605598468343567
0.9599769319492502
0.8985531534551142
0.854141272530197
0.9607049297451775
0.8906824393043221
0.9185635940199833
0.960584573387929
0.9622782446311858
0.9175282793594526
0.9194805194805195
0.9599769319492502
0.8739495798319328
0.8774593029062219
0.9451267686561803
0.8906824393043221
0.8950124933840453
0.9181587451850883
0.9622782446311858
0.9180672268907563
0.9194805194805195
0.9599769319492502
0.8904592139886256
